In [ ]:
import warnings

from keras.datasets import fashion_mnist
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')
%matplotlib inline


In [ ]:
# Some constants
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018

N_TRAIN_SAMPLES = 5000
N_TEST_SAMPLES = 1000

#Model
NO_EPOCHS = 5
BATCH_SIZE = 128


In [ ]:
# Auxiliar functions
def data_preprocessing(x_data, y_data):
    x_data = x_data.astype('float32')
    out_y = to_categorical(y_data, len(np.unique(y_data)))
    out_x = np.expand_dims(x_data, axis=-1)
    
    return out_x, out_y


# Load data

In [ ]:
# load data
(x_train_data, y_train_data), (x_test_data, y_test_data) = fashion_mnist.load_data()

# For this tutorial we will work on a smaller subset
train_idxs = np.random.choice(np.arange(x_train_data.shape[0]), size=N_TRAIN_SAMPLES, replace=False)
test_idxs = np.random.choice(np.arange(x_test_data.shape[0]), size=N_TEST_SAMPLES, replace=False)

x_train_data = x_train_data[train_idxs, :, :]
y_train_data = y_train_data[train_idxs]
x_test_data = x_test_data[test_idxs, :, :]
y_test_data = y_test_data[test_idxs]

# prepare the data
X, y = data_preprocessing(x_train_data, y_train_data)
X_test, y_test = data_preprocessing(x_test_data, y_test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


# Build model

In [ ]:
# Model
model = Sequential()
# Add convolution 2D
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Training data generator
datagen_train = ImageDataGenerator(
    rescale=1./255,  # We also can make a rescale on the data
    horizontal_flip=True,
    rotation_range = 60,
    width_shift_range=0.2,
    height_shift_range=0.2)

# Validation data generator
datagen_val = ImageDataGenerator(
    rescale=1./255)


# Keras Callbacks

A callback is a set of functions to be applied at given stages of the training procedure. You can use callbacks to get a view on internal states and statistics of the model during training. You can pass a list of callbacks (as the keyword argument callbacks) to the .fit() method of the Sequential or Model classes.

## ModelCheckpoint

Save the model after every epoch.

In [ ]:
from keras.callbacks import ModelCheckpoint

callbacks = [
    ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                    monitor='val_loss', 
                    verbose=1, 
                    save_best_only=False, 
                    save_weights_only=False, 
                    mode='auto')
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)



## EarlyStopping

Stop training when a monitored quantity has stopped improving.

In [ ]:
from keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor='val_loss', 
                  min_delta=0, 
                  patience=2, 
                  verbose=1, 
                  mode='auto', 
                  baseline=None, 
                  restore_best_weights=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)


## TensorBoard

TensorBoard basic visualizations.

[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is a visualization tool provided with TensorFlow.

This callback writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.

If you have installed TensorFlow with pip, you should be able to launch TensorBoard from the command line:

```
tensorboard --logdir=/full_path_to_your_logs
```


In [ ]:
from keras.callbacks import TensorBoard

callbacks = [
    TensorBoard(log_dir='./logs', 
                write_graph=True, 
                write_images=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)

## CSVLogger

Callback that streams epoch results to a csv file.

In [ ]:
from keras.callbacks import CSVLogger

callbacks = [
    CSVLogger(filename='train_logs.csv', 
              separator=',', 
              append=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)

## ReduceLROnPlateau

Reduce learning rate when a metric has stopped improving.

Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.

In [ ]:
from keras.callbacks import ReduceLROnPlateau

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', 
                      factor=0.1, 
                      patience=10, 
                      verbose=0, 
                      mode='auto')
]

train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)
